In [1]:
import os
import numpy as np
import time
import random
import matplotlib
import pandas as pd
import jieba
import torch
import torch.optim as optim
import json
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.nn as nn
import torch.nn.functional as F
from keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from transformers import AutoTokenizer, BertModel, BertTokenizer, BertForSequenceClassification, BertConfig, AdamW, get_linear_schedule_with_warmup



In [4]:
device = 'cuda'
MODEL_NAME = 'bert-base-chinese'
num_labels = 3
tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")
max_length = 20
bs = 32
EPOCHS = 2
LR = 1e-5
WARMUP_STEPS = 100

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:

# 初始化BERT的分词器
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

def load_dataset(filepath, max_len):
    labels = []
    sentences = []

    # 读取txt文件
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            # 去掉行末尾的换行符并用逗号分割
            line = line.strip()
            if ',' not in line:
                continue  # 如果没有逗号，跳过
            label, sentence = line.split(',', 1)

            # 将标签和句子添加到相应的列表
            labels.append(int(label))
            sentences.append(sentence)

    input_ids = []
    attention_masks = []

    # 对每个句子进行编码
    for data in sentences:
        encoded_data = tokenizer.encode_plus(
            text=data,                      # 句子
            add_special_tokens=True,        # 添加特殊标记，如[CLS]和[SEP]
            max_length=max_len,             # 截断或填充到最大长度
            padding='max_length',           # 填充到max_length
            return_attention_mask=True,     # 返回attention mask
            truncation=True                 # 截断长句子
        )

        # 将编码后的数据添加到列表中
        input_ids.append(encoded_data.get('input_ids'))
        attention_masks.append(encoded_data.get('attention_mask'))

    # 将列表转换为tensor
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)
    labels = torch.tensor(labels)

    return input_ids, attention_masks, labels


In [6]:
train_dataset = load_dataset('dataset_new.txt', max_len = max_length)
valid_dataset = load_dataset('dataset_test.txt', max_len = max_length)

train_data = TensorDataset(train_dataset[0], train_dataset[1],train_dataset[2])
train_loader = DataLoader(train_data, shuffle=True, batch_size = bs)

val_data = TensorDataset(valid_dataset[0],valid_dataset[1],valid_dataset[2])
val_loader = DataLoader(val_data,shuffle=True, batch_size = bs)

In [7]:
# 定義自己的bert模型
class BertClassifier(nn.Module):
    """Bert Model for Classification Tasks.
    """
    def __init__(self, hidden_dimension):

        super().__init__()

        self.bert = BertModel.from_pretrained(MODEL_NAME)

        embedding_dim = self.bert.config.to_dict()['hidden_size']

        self.LSTM = nn.LSTM(embedding_dim,hidden_dimension,bidirectional=True, batch_first=True)

        self.out = nn.Linear(hidden_dimension * 2, 3)

    def forward(self, input_ids, attention_mask=None):


        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask)

        # Extract the last hidden state of the token `[CLS]` for classification task
        encoded_layers = outputs[0]

        #encoded_layers = encoded_layers.permute(1, 0, 2)

        enc_hiddens, (last_hidden, last_cell) = self.LSTM(encoded_layers)
        output_hidden = torch.cat((enc_hiddens[:,-1, :256],enc_hiddens[:,0, 256:]),dim=-1)
        output_hidden = F.dropout(output_hidden,0.2)

        logits = self.out(output_hidden)

        return logits
def initialize_model(epochs=EPOCHS):
    bert_classifier = BertClassifier(hidden_dimension=256)

    # Tell PyTorch to run the model on GPU
    bert_classifier.to(device)

    # Create the optimizer
    optimizer = AdamW(bert_classifier.parameters(),lr=LR)

    # Set up the learning rate scheduler
    total_steps = len(train_loader) * epochs
    scheduler = get_linear_schedule_with_warmup(optimizer,num_warmup_steps=WARMUP_STEPS,num_training_steps=total_steps)
    return bert_classifier, optimizer, scheduler

In [8]:
loss_fn = nn.CrossEntropyLoss()
#定義訓練跟評估
def train(model, train_dataloader, val_dataloader=None, epochs=EPOCHS , evaluation=False):
    """Train the BertClassifier model.
    """
    # Start training loop
    print("Start training...\n")
    for epoch_i in range(epochs):
        # Print the header of the result table
        print(f"{'Epoch':^7} | {'Batch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {'Val Acc':^9} | {'Elapsed':^9}")
        print("-"*70)

        # Measure the elapsed time of each epoch
        t0_epoch, t0_batch = time.time(), time.time()

        # Reset tracking variables at the beginning of each epoch
        total_loss, batch_loss, batch_counts = 0, 0, 0

        # Put the model into the training mode
        model.train()

        # For each batch of training data...
        for s,batch in enumerate(train_dataloader):
            batch_counts +=1
            # Load batch to GPU
            b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

            logits = model(b_input_ids, b_attn_mask)
            # Compute loss and accumulate the loss values
            loss = loss_fn(logits.view(-1, 3), b_labels.view(-1))
            batch_loss += loss.item()
            total_loss += loss.item()

            # Zero out any previously calculated gradients
            model.zero_grad()

            # return loss, logits
            # Perform a backward pass to calculate gradients
            loss.backward()
            optimizer.step()
            scheduler.step()

            # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Print the loss values and time elapsed for every 20 batches
            if (s % 20 == 0 and s != 0) or (s == len(train_dataloader) - 1):
                # Calculate time elapsed for 20 batches
                time_elapsed = time.time() - t0_batch

                # Print training results
                print(f"{epoch_i + 1:^7} | {s:^7} | {batch_loss / batch_counts:^12.6f} | {'-':^10} | {'-':^9} | {time_elapsed:^9.2f}")

                # Reset batch tracking variables
                batch_loss, batch_counts = 0, 0
                t0_batch = time.time()

        # Calculate the average loss over the entire training data
        avg_train_loss = total_loss / len(train_dataloader)

        print("-"*70)
        if evaluation == True:
            val_loss, val_accuracy = evaluate(model, val_dataloader)
            time_elapsed = time.time() - t0_epoch
            print(f"{epoch_i + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}")
            print("-"*70)
        print("\n")

    print("Training complete!")

def evaluate(model, val_dataloader):
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_loss = []

    # For each batch in our validation set...
    for batch in val_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)

        # Compute loss
        loss = loss_fn(logits, b_labels)
        val_loss.append(loss.item())

        # Get the predictions
        preds = torch.argmax(logits, dim=1).flatten()

        # Calculate the accuracy rate
        accuracy = (preds == b_labels).cpu().numpy().mean() * 100
        val_accuracy.append(accuracy)

    # Compute the average accuracy and loss over the validation set.
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)

    return val_loss, val_accuracy


In [9]:
bert_classifier, optimizer, scheduler = initialize_model(epochs=EPOCHS)
train(bert_classifier, train_loader, val_loader, epochs=EPOCHS, evaluation=True)

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Start training...

 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------
   1    |   20    |   1.067690   |     -      |     -     |   4.35   
   1    |   40    |   0.949167   |     -      |     -     |   2.49   
   1    |   60    |   0.802394   |     -      |     -     |   2.51   
   1    |   80    |   0.667499   |     -      |     -     |   2.53   
   1    |   100   |   0.469111   |     -      |     -     |   2.53   
   1    |   120   |   0.308112   |     -      |     -     |   2.53   
   1    |   140   |   0.282987   |     -      |     -     |   2.48   
   1    |   160   |   0.234066   |     -      |     -     |   2.51   
   1    |   180   |   0.181462   |     -      |     -     |   2.53   
   1    |   200   |   0.215741   |     -      |     -     |   2.52   
   1    |   220   |   0.198554   |     -      |     -     |   2.58   
   1    |   240   |   0.189979   |     -      |     -     |   2.59   


In [11]:
torch.save(bert_classifier.state_dict(), 'bert_lstm.ckpt')

print('开始测试...')
bert_classifier.eval()
test_result = []
for input_ids, attention_mask, labels in zip(valid_dataset[0], valid_dataset[1], valid_dataset[2]):
    b_input_ids = input_ids.unsqueeze(0).to(device)
    b_attn_mask = attention_mask.unsqueeze(0).to(device)
    b_labels = labels.unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = bert_classifier(b_input_ids, attention_mask=b_attn_mask)
        pre = outputs.argmax(dim=1)
        tokens = [tokenizer.convert_ids_to_tokens(id.item()) for id in b_input_ids[0]]
        test_result.append([b_labels.item(), pre.item(), tokens])
       #b_labels.item()：获取真实标签的值。pre.item()：获取模型预测的值。tokenizer.convert_ids_to_tokens(b_input_ids)：将输入的 token IDs 转换回原始文本形
# 写入csv文件
df = pd.DataFrame(test_result)
df.to_csv('test_result.csv',index=False, header=['real', 'predict','text'])

开始测试...


In [13]:
df = pd.read_csv('test_result.csv')
print(df)

     real  predict                                               text
0       1        1  ['[CLS]', '好', '吧', '移', '动', '年', '优', '惠', '...
1       1        1  ['[CLS]', '好', '听', '歌', '曲', '名', '单', '[SEP]...
2       1        1  ['[CLS]', '常', '州', '市', '贸', '中', '心', '干', '...
3       0        0  ['[CLS]', '无', '聊', '时', '候', '还', '可', '以', '...
4       0        0  ['[CLS]', '就', '想', '知', '道', '怎', '么', '那', '...
..    ...      ...                                                ...
119     1        1  ['[CLS]', '里', '面', '阿', '狸', '可', '爱', '[SEP]...
120     0        0  ['[CLS]', '没', '用', '系', '统', '设', '置', '[SEP]...
121     1        1  ['[CLS]', '可', '爱', '自', '恋', '吧', '好', '困', '...
122     1        1  ['[CLS]', '通', '知', '求', '主', '动', '活', '跃', '...
123     0        0  ['[CLS]', '我', '试', '着', '那', '样', '做', '可', '...

[124 rows x 3 columns]
